In [1]:
print("OK!")

OK!


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os

c:\Users\ASUS\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "pcsk_6bCDku_SYnG1QeQTGJheCLLSBdf4i9J9CrXBmUGwhsB5QhBmg6BQdZoUhUSrteQAPYoXbS"

In [4]:
#Extract data from PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_spliter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_spliter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 39994


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21368\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
query_result = embeddings.embed_query("Hello world")
print("lenght", len(query_result))

lenght 384


In [30]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_6bCDku_SYnG1QeQTGJheCLLSBdf4i9J9CrXBmUGwhsB5QhBmg6BQdZoUhUSrteQAPYoXbS")
index = pc.Index("medical-chatbot")

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], index_name="medical-chatbot", embedding=embeddings)

In [ ]:
#If we already have an index we can load it like this 
docsearch = PineconeVectorStore.from_existing_index("medical-chatbot", embeddings)

# Query
query = "what are allergies"

# Perform similarity search
docs = docsearch.similarity_search(query, k=3)

print("Result:", docs)

Result: [Document(page_content='‘‘What’s New in: Asthma and Allergic Rhinitis.’’Pulse\n(September 20, 2004): 50.\nRichard Robinson\nTeresa G. Odle\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune\nsystem that occur in response to otherwise harmless\nsubstances.\nDescription\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, or\nmore than one in every five people, suffer from some\nform of allergy, with similar proportions throughout'), Document(page_content='KEY TERMS\nAllergen— A substance capable of inducing an\nallergic response.\nAllergic reaction— An immune system reaction to\na substance in the environment; symptoms include\nrash, inflammation, sneezing, itchy watery eyes,\nand runny nose.\nConjunctiva— The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.\nEdema— A condition where tissues contain exces-\nsive fluid.\nMeibomian gland— Oil-producing glands in the\neyelids that op

In [34]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [35]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [36]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [37]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21368\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query": user_input})


Response :  Antimalarial refers to a type of drug that is used to treat malaria, which is a disease caused by a parasite that is transmitted through the bite of an infected mosquito. Antimalarial drugs work by killing the parasite or preventing it from reproducing in the body. Examples of antimalarial drugs include chloroquine (Aralen), mefloquine (Lariam), primaquine, pyrimethamine (Daraprim), and quinine.
